In [58]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from image_preprocessing import merge_df
from torch.utils.tensorboard import SummaryWriter

In [109]:
class ImageDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.data = pd.read_pickle(os.path.realpath(os.path.join(os.path.dirname("__file__"),'Image_data.pkl')))
        self.category_map = {
            "Home & Garden": 1,
            "Baby & Kids Stuff": 2,
            "DIY Tools & Materials": 3,
            "Music, Films, Books & Games": 4,
            "Phones, Mobile Phones & Telecoms": 5,
            "Clothes, Footwear & Accessories": 6,
            "Other Goods": 7,
            "Health & Beauty" :8,
            "Sports, Leisure & Travel": 9,
            "Appliances": 10,
            "Computers & Software": 11,
            "Office Furniture & Equipment": 12,
            "Video Games & Consoles": 13
            }

    def __getitem__(self, index):
        example = self.data.iloc[index]
        features = example[-1]
        label = example[0]
        return (torch.tensor(features).float(), label)
    
    def decode(self, encoded_value):
        numbers = list(self.category_map.values())
        words = list(self.category_map.keys())
        position = numbers.index(encoded_value)
        decoded = words[position]
        return decoded
    
    def encode(self, decoded_value):
        numbers = list(self.category_map.values())
        words = list(self.category_map.keys())
        position = words.index(decoded_value)
        encoded = numbers[position]
        return encoded
    
    def __len__(self):
        return len(self.data)

In [120]:
dataset = ImageDataset()
train_loader = DataLoader(dataset, batch_size=16, shuffle=True)

In [145]:
def train(model, epochs=10):
    optimiser = torch.optim.SGD(model.parameters(), lr=0.001)

    writer = SummaryWriter()
    batch_idx = 0

    for epoch in range(epochs):
        for batch in train_loader:
            features, labels = batch
            prediction = model(features)
            loss = F.cross_entropy(prediction, labels.long())
            print('Loss:', loss.item())
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()
            writer.add_scalar('Loss', loss.item(), batch_idx)
            batch_idx += 1

In [146]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = torch.nn.Sequential(
            torch.nn.Conv2d(512, 8, 1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(8, 16, 1),
            torch.nn.Flatten(),
            torch.nn.Linear(8192, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 14),
            torch.nn.Softmax()
        )
    
    def forward(self, X):
        return self.layers(X)

In [147]:
model = CNN()
train(model)

Loss: 2.711724281311035
Loss: 2.6929194927215576
Loss: 2.7269394397735596
Loss: 2.654729127883911
Loss: 2.620720624923706
Loss: 2.753325939178467
Loss: 2.683917284011841
Loss: 2.743302822113037
Loss: 2.720398426055908
Loss: 2.736598253250122
Loss: 2.729381561279297
Loss: 2.662950038909912
Loss: 2.744239330291748
Loss: 2.697819948196411
Loss: 2.603066921234131
Loss: 2.728046417236328
Loss: 2.500562906265259
Loss: 2.7175498008728027
Loss: 2.7452211380004883
Loss: 2.6624112129211426
Loss: 2.6863415241241455
Loss: 2.596294641494751
Loss: 2.6778786182403564
Loss: 2.587946653366089
Loss: 2.6528093814849854
Loss: 2.677567720413208
Loss: 2.6219611167907715
Loss: 2.60526704788208
Loss: 2.6237287521362305
Loss: 2.676666498184204
Loss: 2.7439141273498535
Loss: 2.7296550273895264
Loss: 2.6860828399658203
Loss: 2.7055654525756836
Loss: 2.6573994159698486
Loss: 2.658024311065674
Loss: 2.740854024887085
Loss: 2.6781716346740723
Loss: 2.6259753704071045
Loss: 2.729231357574463
Loss: 2.7051334381103516